In [1]:
##어떤 클래스를 사용할지 
## 각 클래스를 사용해서 5개 기사에 대해서 비교 후 선정하는 걸로
## 클래스는 okt, komoran, mecab, kkma,Hannanum

In [1]:
from konlpy.tag import Okt
from konlpy.tag import Komoran
from konlpy.tag import Kkma
from konlpy.tag import Hannanum

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

import pandas as pd
import re  

from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
from numpy import dot
from numpy.linalg import norm 
import math

In [2]:
def cleansing(text):  
    new_text=' '.join(text.split()) #공백 제거
    pattern = '[0-9]'
    new_text = re.sub(pattern=pattern,repl=' ',string=new_text)
    
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' #이메일 주소 제거
    new_text = re.sub(pattern=pattern, repl='', string=text)
    
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:\da-fA-F]{2}))+'  # url 제거
    new_text = re.sub(pattern=pattern,repl=' ',string=new_text)

    pattern = '<[^>]*>'                                          # html tag 제거
    new_text = re.sub(pattern=pattern,repl=' ',string=new_text)

    pattern = '[\r|\n]'                                          # \r, \n 제거
    new_text = re.sub(pattern=pattern,repl=' ',string=new_text)

    pattern = '[^\w\s]'                                          # 특수기호 제거
    new_text = re.sub(pattern=pattern,repl=' ',string=new_text)

    pattern = re.compile(r'\s+')                                 # 이중 space 제거
    new_text = re.sub(pattern=pattern,repl=' ',string=new_text)
    
    
    ##불용어 처리
    stopword="\d|급$|씨$|월$|일$|더불어민주당|새누리당|야당|여당|사진|이날|_|기자$|위|등" 

    word_tokens=word_tokenize(new_text)
    data_word=[x for x in word_tokens if not re.search(stopword,x)]
    new_text=" ".join(data_word)
    

    return(new_text)

In [3]:
def corpus(tmp_list):
    return ' '+' '.join(tmp_list)

In [6]:
def pearson(tf_idf_matrix):
    array=tf_idf_matrix.toarray()
    
    m=len(array[0])
    TFa=sum(array[0])
    TFb=sum(array[1])

    sum_w=dot(array[0],array[1])

    sum_wa2=sum(array[0]**2); sum_wb2=sum(array[1]**2)
    norma=m*sum_wa2-TFa**2; normb=m*sum_wb2-TFb**2

    up=(m*sum_w)-(TFa*TFb)
    down=math.sqrt(float(norma*normb))

    return up/down

In [7]:
Random_Docs=pd.read_excel('D:\\KMI-Project_tmp\\Data\\Sentimental\\RandomDocs15.xlsx') # 랜덤 기사 문서
origin_list=[]
okt=Okt(); okt_doc=''
komo=Komoran(); komo_doc=''
kkma=Kkma(); kkma_doc=''
han=Hannanum(); han_doc=''
package_list=['original','okt','komo','kkma','han'] #사용된 한국어 패키지

pearson_df=pd.DataFrame() 
total_docs=[];

for index,row in Random_Docs.iterrows(): #랜덤 기사의 순서대로 iteration 진행 
    
    doc=cleansing(row['본문']); origin_list.append(row['키워드'])
    #original 본문에 대해서 각 패키지를 활용하여 토큰화 진행 
    okt_doc+=corpus(okt.nouns(doc)); komo_doc+=corpus(komo.nouns(doc))
    kkma_doc+=corpus(kkma.nouns(doc)); han_doc+=corpus(han.nouns(doc))

origin_doc=corpus(origin_list)
total_docs.append(origin_doc); total_docs.append(okt_doc); total_docs.append(komo_doc)
total_docs.append(kkma_doc); total_docs.append(han_doc)
    
for index in range(5):
    comp_doclist=[]
    comp_doclist.append(total_docs[0]); comp_doclist.append(total_docs[index])
        #비교할 문서리스트 선언: total_docs[0]: 빅카인즈의 키워드, total_docs[index]: 각각 순서대로 패키지의 산출물 
    tfidf_matrix=TfidfVectorizer().fit_transform(comp_doclist) 
        #tfidf 행렬을 이용(논문 참고)
    pearson_df=pearson_df.append(pd.Series([package_list[index],pearson(tfidf_matrix)],index=['Version','Value']),ignore_index=True)

display(pearson_df)

,Value,Version
0,1.000000,original
1,0.591504,okt
2,0.621768,komo
3,0.327784,kkma
4,0.593158,han


In [ ]:
## komo 패키지 사용~